<img src="./img/hpe_logo.png" alt="HPE Logo" width="125">

<h1>HPE ML Platform Workshop - Inferencing</h1>

<img src='img/platform_step03_deployment.png' width='1200'/>

<h3>Import modules and define functions</h3>

In [ ]:
# Imports
import json
import uuid
import torch
import base64
import requests
import numpy as np
import matplotlib.pyplot as plt
from configparser import ConfigParser

from PIL import Image

<h3>Step 1: Setting up connection details to KServe and defining support functions</h3>

In [ ]:
# Load Data and Set Variables
config_obj = ConfigParser()
config_obj.read("./utils/config.ini")

model_name = config_obj['PDK_INFO']['model_name']
ingress_host = config_obj['PDK_INFO']['ingress_host']
ingress_port = config_obj['PDK_INFO']['ingress_port']

In [ ]:
service_hostname = "brain-mri-ws-deploy.models.example.com"

In [ ]:
# Function to caluclate intersection over union of prediction
def iou(pred, label):
    intersection = (pred * label).sum()
    union = pred.sum() + label.sum() - intersection
    if pred.sum() == 0 and label.sum() == 0:
        return 1
    return intersection / union

# Function to create tensor for image and mask
def PairedToTensor(sample):
    img, mask = sample
    img = np.array(img)
    mask = np.expand_dims(mask, -1)
    img = np.moveaxis(img, -1, 0)
    mask = np.moveaxis(mask, -1, 0)
    img, mask = torch.FloatTensor(img), torch.FloatTensor(mask)
    img = img/255
    mask = mask/255
    return img, mask

In [ ]:
def create_request(image, mask):
    # Create tuple
    sample = (image, mask)

    # Create tensors from tuple
    tensor_sample = PairedToTensor(sample)

    # Create JSON payload for request
    data = np.array(tensor_sample[0])
    data_shape = list(data.shape)
    request = {
        "inputs": [{
            "name": str(uuid.uuid4()),
            "shape": data_shape,
            "datatype": "FP32",
            "data": np.round(data, 4).tolist()
        }]
    }
    return tensor_sample, request

In [ ]:
def render_image(tensor_sample):
    # Show image that will be submitted
    plt.figure(figsize=(7,7))
    plt.title(f'Submitted Image: ')
    plt.imshow(tensor_sample[0].permute(1, 2, 0))

In [ ]:
def get_prediction(image, mask, tensor_sample, request):
    # Create GRPC request for Prediction (header, URL, payload)
    url = str("http://") + str(ingress_host) + ":" + str(ingress_port) + "/v2/models/" + str(model_name) + "/infer"
    headers = {'Host': service_hostname, "Content-Type": "application/json"}
    payload = json.dumps(request)
    
   # Submit request, extract prediction in JSON, transform to Tensor
    response = requests.post(url, data=payload, headers=headers)
    output = response.json()

    shape = [1,256,256]
    values = output["outputs"][0]["data"]
    output = torch.Tensor(np.array(values).reshape(shape)) 
    
    # # Display groundtruth and prediction mask, call iou function and display iou
    # f, axarr = plt.subplots(1,2, figsize=(15, 15))
    # axarr[0].imshow(tensor_sample[1].permute(1, 2, 0), alpha=0.4)
    # axarr[0].title.set_text(f'Mask (Ground Truth):')
    # axarr[1].imshow(output.permute(1, 2, 0), alpha=0.4)
    # axarr[1].title.set_text(f'Mask (Prediction):')
    # print(f'Intersection over Union (IoU): {iou(output, tensor_sample[1])}')
    
    # Display groundtruth and prediction overlaid on submitted image, call iou function and display iou
    f, axarr = plt.subplots(1,2, figsize=(15, 15))
    axarr[0].imshow(tensor_sample[0].permute(1, 2, 0))
    axarr[0].imshow(tensor_sample[1].permute(1, 2, 0), alpha=0.4)
    axarr[0].title.set_text(f'Full Image (Ground Truth):')
    axarr[1].imshow(tensor_sample[0].permute(1, 2, 0))
    axarr[1].imshow(output.permute(1, 2, 0), alpha=0.4)
    axarr[1].title.set_text(f'Full Image (Prediction):')
    print(f'Intersection over Union (IoU): {iou(output, tensor_sample[1])}')    

<h3>Step 2: Generate a prediction for a single image</h3>

In [ ]:
# Load image and mask
image = Image.open("../images/brain/TCGA_CS_6290_20000917/TCGA_CS_6290_20000917_10.tif")
mask = Image.open("../images/brain/TCGA_CS_6290_20000917/TCGA_CS_6290_20000917_10_mask.tif")

In [ ]:
tensor_sample, request = create_request(image, mask)

In [ ]:
render_image(tensor_sample)

In [ ]:
get_prediction(image, mask, tensor_sample, request)

<h3>Step 3: Generate a prediction for multiple images</h3>

In [ ]:
input_list = [
    ["../images/brain/TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_13.tif", "../images/brain/TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_13_mask.tif"],
    ["../images/brain/TCGA_CS_6669_20020102/TCGA_CS_6669_20020102_14.tif", "../images/brain/TCGA_CS_6669_20020102/TCGA_CS_6669_20020102_14_mask.tif"],
    ["../images/brain/TCGA_CS_5393_19990606/TCGA_CS_5393_19990606_10.tif", "../images/brain/TCGA_CS_5393_19990606/TCGA_CS_5393_19990606_10_mask.tif"],
    ["../images/brain/TCGA_CS_4942_19970222/TCGA_CS_4942_19970222_11.tif", "../images/brain/TCGA_CS_4942_19970222/TCGA_CS_4942_19970222_11_mask.tif"],
    ["../images/brain/TCGA_CS_6665_20010817/TCGA_CS_6665_20010817_13.tif", "../images/brain/TCGA_CS_6665_20010817/TCGA_CS_6665_20010817_13_mask.tif"],
]

In [ ]:
for item in input_list:
    image = Image.open(item[0])
    mask = Image.open(item[1])
    tensor_sample, request = create_request(image, mask)
    get_prediction(image, mask, tensor_sample, request)